# Explanatory Data Analysis on UCF Parking Garage Data

## Table of Contents

1. [**Introduction**](#1)
    - Project Description
    - Data Description
2. [**Acquiring and Loading Data**](#2)
	- Importing Libraries and Notebook Setup
    - Loading Data
    - Basic Data Exploration
    - Areas to Fix
3. [**Data Proprocessing**](#3)
4. [**Data Analysis**](#4)
5. [**Conclusion**](#5)
    - Insights
    - Suggestions
    - Possible Next Steps
6. [**Epilogue**](#6) 
    - References
    - Versioning

---

# 1

## Introduction

### Project Description

**Goal/Purpose:** 

This project, parkUCF, aims to facilitate student parking at the University of Central Florida's parking garages. The objective is to deploy an application that leverages machine learning predictions to display information and insights to students regarding the expected garage occupancy for the day. This notebook will outline and document the process of explanatory data analysis, and its findings will be instrumental to the completion of parkUCF.

<p>&nbsp;</p>

**Questions to be Answered:**

- Do time formatting issues cause problems in the dataset?
- What are the busiest and least busy garages and days of the week?
- How similar are occupancy trends from day to day?

<p>&nbsp;</p>

**Assumptions/Methodology/Scope:** 

Assumptions:

- All timestamps are timezone-aware/in UTC
- The device which scans license plates returns accurate vehicle counts
- The number of occupied spots represents the number of vehicles inside the garage

Methodology/processing steps:

- Data inspection, cleaning and preprocessing
- Multivariate analysis
- Feature engineering
- Visualization

Scope:

- Exploring and understanding the dataset
- Only the data provided is analyzed

<p>&nbsp;</p>

### Data Description

**Content:** 

This dataset is a JSON file of 25989 data points which contains parking garage data from 5 UCF parking garages (garages A, B, C, D, and H). 

<p>&nbsp;</p>

**Description of Attributes:** 

Here you can describe what each column represents.

| Column  | Description |
| :------ | :---------- |
| name    | Name of the garage|
|available| total - amount of vehicles in garage|
|occupied | Amount of vehicles in garage|
|total    | Total amount of spots in garage |
|occupancy_rate|occupied/available|
|event_reserved|Amount of spots reserved for an event|
|event_name|The name of the event|
|timestamp| The timestamp in UTC of the datapoint|

<p>&nbsp;</p>

---

# 2

## Acquiring and Loading Data
### Importing Libraries and Notebook Setup

In [2]:
# Data manipulation
import datetime
import numpy as np
import pandas as pd
import pandas.api.types as ptypes
from skimpy import skim, clean_columns

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Pandas settings
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 60
pd.options.display.float_format = '{:,.3f}'.format

# Visualization settings
from matplotlib import rcParams
plt.style.use('fivethirtyeight')
rcParams['figure.figsize'] = (16, 5)   
rcParams['axes.spines.right'] = False
rcParams['axes.spines.top'] = False
rcParams['font.size'] = 12
# rcParams['figure.dpi'] = 300
rcParams['savefig.dpi'] = 300
plt.rc('xtick', labelsize=11)
plt.rc('ytick', labelsize=11)
custom_palette = ['#003f5c', '#444e86', '#955196', '#dd5182', '#ff6e54', '#ffa600']
custom_hue = ['#004c6d', '#346888', '#5886a5', '#7aa6c2', '#9dc6e0', '#c1e7ff']
custom_divergent = ['#00876c', '#6aaa96', '#aecdc2', '#f1f1f1', '#f0b8b8', '#e67f83', '#d43d51']
sns.set_palette(custom_palette)
%config InlineBackend.figure_format = 'retina'

### Loading Data

In [3]:
df = pd.read_json("../data/raw/parking_data.json")

### Basic Data Exploration

In [4]:
print(f"Rows count: {df.shape[0]}\nColumns count: {df.shape[1]}")

Rows count: 26799
Columns count: 8


In [5]:
df.head()

,name,available,occupied,total,occupancy_rate,timestamp,event_reserved,event_name
0,Garage A,1647,0,1647,0.000,2025-09-22 01:45:17.937,NaN,NaN
1,Garage B,1289,0,1289,0.000,2025-09-22 01:45:17.937,NaN,NaN
2,Garage C,1852,0,1852,0.000,2025-09-22 01:45:17.937,NaN,NaN
3,Garage H,1340,0,1340,0.000,2025-09-22 01:45:17.937,NaN,NaN
4,Garage A,1647,0,1647,0.000,2025-09-22 01:47:17.570,NaN,NaN


In [6]:
df.tail()

,name,available,occupied,total,occupancy_rate,timestamp,event_reserved,event_name
26794,Garage A,1399,248,1647,0.151,2025-10-01 02:07:16.732,0.000,None
26795,Garage B,1289,0,1289,0.000,2025-10-01 02:07:16.732,0.000,None
26796,Garage C,1789,63,1852,0.034,2025-10-01 02:07:16.732,0.000,None
26797,Garage D,680,599,1289,0.465,2025-10-01 02:07:16.732,0.000,None
26798,Garage H,1340,0,1340,0.000,2025-10-01 02:07:16.732,0.000,None


#### Check Data Types

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26799 entries, 0 to 26798
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            26799 non-null  object        
 1   available       26799 non-null  int64         
 2   occupied        26799 non-null  int64         
 3   total           26799 non-null  int64         
 4   occupancy_rate  26799 non-null  float64       
 5   timestamp       26799 non-null  datetime64[ns]
 6   event_reserved  20196 non-null  float64       
 7   event_name      1241 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1.6+ MB


- `name`,`event_name` are **strings**.
- `available`, `occupied`, `total`, `event_reserved` are **integers**.
- `occupancy_rate` is a **float**.
- `timestamp` is a **datetime** object.

#### Check Missing Data

In [8]:
missing_percent = df.isna().mean().sort_values(ascending=False)
print('---- Percentage of Missing Values (%) -----')
if missing_percent.sum():
    print(missing_percent[missing_percent > 0] * 100)
else:
    print('None')

---- Percentage of Missing Values (%) -----
event_name       95.369
event_reserved   24.639
dtype: float64


#### Check for Duplicate Rows

In [9]:
print(f"No. of entirely duplicated rows: {df.duplicated().sum()}")

df[df.duplicated()]

No. of entirely duplicated rows: 1618


,name,available,occupied,total,occupancy_rate,timestamp,event_reserved,event_name
403,Garage A,1636,11,1647,0.007,2025-09-22 10:53:16.563,NaN,NaN
404,Garage B,1118,171,1289,0.133,2025-09-22 10:53:16.563,NaN,NaN
405,Garage C,1823,29,1852,0.016,2025-09-22 10:53:16.563,NaN,NaN
406,Garage D,1270,9,1289,0.007,2025-09-22 10:53:16.563,NaN,NaN
407,Garage H,1225,115,1340,0.086,2025-09-22 10:53:16.563,NaN,NaN
...,...,...,...,...,...,...,...,...
26614,Garage A,1312,335,1647,0.203,2025-10-01 00:53:16.762,0.000,None
26615,Garage B,1025,264,1289,0.205,2025-10-01 00:53:16.762,0.000,None
26616,Garage C,1562,290,1852,0.157,2025-10-01 00:53:16.762,0.000,None
26617,Garage D,637,642,1289,0.498,2025-10-01 00:53:16.762,0.000,None


#### Check Uniqueness of Data

In [10]:
# Print the percentage similarity of values (the lower %, the better)
num_unique = df.nunique().sort_values()
print('---- Percentage Similarity of Values (%) -----')
print(100/num_unique)

---- Percentage Similarity of Values (%) -----
event_reserved   50.000
event_name       33.333
name             20.000
total            20.000
occupancy_rate    0.099
occupied          0.056
available         0.054
timestamp         0.020
dtype: float64


#### Check Data Range

In [11]:
# Print summary statistics
df.describe(include='all')
skim(df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 26799  │ │ int64       │ 3     │                                                          │
│ │ Number of columns │ 8      │ │ string      │ 2     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 2     │                                                          │
│                                │ datetime64  │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┓  │
│ ┃ column         ┃ NA   ┃ NA %             ┃ mean   ┃ sd     ┃ p0   ┃ p25   ┃ p50   ┃ p75   ┃ p100  ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━┩  │
│ │ available      │    0 │                0 │   1007 │  512.8 │    0 │   553 │  1191 │  1340 │  1852 │ ▃▅▃▃▇▅ │  │
│ │ occupied       │    0 │                0 │  471.3 │  501.7 │    0 │    16 │   288 │   848 │  1876 │ ▇▂▂▂▁▁ │  │
│ │ total          │    0 │                0 │   1480 │  225.2 │ 1289 │  1289 │  1340 │  1647 │  1852 │ ▇  ▃ ▃ │  │
│ │ occupancy_rate │    0 │                0 │ 0.3171 │ 0.3239 │    0 │ 0.011 │ 0.191 │ 0.606 │ 1.013 │ ▇▁▂▂▂▁ │  │
│ │ event_reserved │ 6603 │ 24.6389790663830 │ 0.4134 │  3.188 │    0 │     0 │     0 │     0 │    25 │   ▇    │  │
│ │                │      │               72 │        │        │      │       │       │       │       │        │  │
│ └────────────────┴──────┴──────────────────┴────────┴────────┴──────┴───────┴───────┴───────┴───────┴────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓  │
│ ┃ column        ┃ NA   ┃ NA %    ┃ first                        ┃ last                        ┃ frequency    ┃  │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩  │
│ │ timestamp     │    0 │       0 │  2025-09-22 01:45:17.937000  │ 2025-10-01 02:07:16.732000  │ None         │  │
│ └───────────────┴──────┴─────────┴──────────────────────────────┴─────────────────────────────┴──────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓  │
│ ┃          ┃       ┃          ┃          ┃          ┃          ┃          ┃ chars per ┃ words    ┃ total     ┃  │
│ ┃ column   ┃ NA    ┃ NA %     ┃ shortest ┃ longest  ┃ min      ┃ max      ┃ row       ┃ per row  ┃ words     ┃  │
│ ┡━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩  │
│ │ name     │     0 │        0 │ Garage A │ Garage A │ Garage A │ Garage H │         8 │        2 │     53598 │  │
│ │ event_na │ 25558 │ 95.36923 │ Startup  │ Real     │ CREOL    │ Startup  │      32.6 │     0.23 │      6110 │  │
│ │ me       │       │ 01951565 │ Fest     │ Estate   │ 

### Areas to Fix
**Duplicate Rows**
- Most 'duplicates' are most likely merely results of the occupancy rate not changing much in certain dormant periods, so no need to fix this during preprocessing

**Redundant Columns**
- `occupied`, `available`, and `total` are redundant since we've already extracted `occupancy_rate` during data extraction.

**Irrelevant Columns**
- `event_name`, `event_reserved` are missing for most of the dataset and are not needed during modeling.

**Timestamp Format Inconsistency**
- Though timestamps are consistently timezone-aware/in UTC, and despite pandas handling this gracefully (no discrepancies within dataframe), some timestamps are in ISO format while others are not. We will be using them as an all-in-one time feature for the model.

---

# 3

## Data Preprocessing

### Drop Redundant Columns

In [ ]:
cols_to_drop = ['occupied', 'available', 'total']
df.drop(columns=cols_to_drop, axis=1, inplace=True)

In [13]:
assert all(col not in df.columns for col in cols_to_drop)

### Drop Irrelevant Columns

In [ ]:
cols_to_drop = ['event_name', 'event_reserved']
df.drop(columns=cols_to_drop, axis=1, inplace=True)

In [ ]:
# assert indicates preprocessing is done
assert all(col not in df.columns for col in cols_to_drop)

### Handling Timestamp Format Inconsistency

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

### Feature Engineering / Transformation

In [20]:
# # Get unique values of interested columns
# cols = []
# pd.unique(df[cols].values.ravel('k'))  # argument 'k' lists the values in the order of the cols 

In [21]:
# # Create custom function
# # Google style docstrings
# # https://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html
# def custom_function(param1: int, param2: str) -> bool:
#     """Example function with PEP 484 type annotations.

#     Args:
#         param1: The first parameter.
#         param2: The second parameter.

#     Returns:
#         The return value. True for success, False otherwise.

#     """

In [22]:
# # Apply function to multiple columns
# cols = []
# df_updated = df.copy()
# df_updated[cols] = df_updated[cols].applymap(custom_function)

# # Create new aggregated boolean column
# df_updated['bool'] = df_updated[cols].any(axis=1, skipna=False)

---

# 4

## Data Analysis

Here is where your analysis begins. You can add different sections based on your project goals.

### Exploring `Column Name`

In [23]:
# Code and visualization

**Observations**
- Ob 1
- Ob 2
- Ob 3

---

# 5

## Conclusion

### Insights 
State the insights/outcomes of your project or notebook.

### Suggestions

Make suggestions based on insights.

### Possible Next Steps
Areas to expand on:
- (if there is any)

---

# 6

## Epilogue

### References

This is how we use inline citation[<sup id="fn1-back">[1]</sup>](#fn1).

[<span id="fn1">1.</span>](#fn1-back) _Author (date)._ Title. Available at: https://website.com (Accessed: Date). 

> Use [https://www.citethisforme.com/](https://www.citethisforme.com/) to create citations.

### Versioning
Notebook and insights by (author).
- Version: 1.5.0
- Date: 2023-05-15

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b689e16b-e36c-4f8c-b17a-b3e876352669' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>